# Generator

**CS1302 Introduction to Computer Programming**
___

In [ ]:
%reload_ext divewidgets

## Recursion

Consider computing the [Fibonacci number](https://en.wikipedia.org/wiki/Fibonacci_number) of order $n$:

$$
F_n := 
\begin{cases}
F_{n-1}+F_{n-2} & n>1 \kern1em \text{(recurrence)}\\
1 & n=1 \kern1em \text{(base case)}\\
0 & n=0 \kern1em \text{(base case)}.
\end{cases}$$
Fibonacci numbers have practical applications in generating [pseudorandom numbers](https://en.wikipedia.org/wiki/Lagged_Fibonacci_generator).

**Can we define the function by calling the function itself?**

Try stepping through such a function below to see how it works:

In [ ]:
%%optlite -r -h 450
def fibonacci(n):
    if n > 1:
        return fibonacci(n - 1) + fibonacci(n - 2)  # recursion
    elif n == 1:
        return 1
    else:
        return 0


print(fibonacci(2))

::::{important}

A function that calls itself (*recurs*) is called a [*recursion*](https://en.wikipedia.org/wiki/Recursion_(computer_science)).
::::

::::{exercise}
:label: ex:gcd 

Write a function `gcd` that implements the [Euclidean algorithm for the greatest common divisor](https://en.wikipedia.org/wiki/Euclidean_algorithm): 

$$\operatorname{gcd}(a,b)=\begin{cases}a & b=0\\ \operatorname{gcd}(b, a\operatorname{mod}b) & \text{otherwise.} \end{cases}$$

::::

In [ ]:
%%optlite -r -h 550
def gcd(a, b):
    # YOUR CODE HERE
    raise NotImplementedError()


print(gcd(3 * 5, 5 * 7)) # gcd = ?

**Is recursion strictly necessary?**

E.g., the following computes the Fibonacci number using a while loop instead.

In [ ]:
%%optlite -r -h 550
def fibonacci_iteration(n):
    if n > 1:
        _, F = 0, 1  # next two Fibonacci numbers
        while n > 1:
            _, F, n = F, F + _, n - 1
        return F
    elif n == 1:
        return 1
    else:
        return 0


fibonacci_iteration(3)

In [ ]:
# more tests
for n in range(5):
    assert fibonacci(n) == fibonacci_iteration(n)

::::{exercise}
:label: ex:recur2iter 

We can always convert a recursion to an iteration. Why?

:::{hint}
:class: dropdown

See the [recursion vs interaction](https://en.wikipedia.org/wiki/Recursion_(computer_science)#Recursion_versus_iteration).

:::
::::

YOUR ANSWER HERE

::::{exercise}
:label: ex:gcd_iteration 

Implement `gcd_iteration` using a while loop instead of recursion.

::::{hint}

See [tail recursion](https://en.wikipedia.org/wiki/Recursion_(computer_science)#Tail-recursive_functions).
::::

In [ ]:
%%optlite -r -h 550
def gcd_iteration(a, b):
    # YOUR CODE HERE
    raise NotImplementedError()


gcd_iteration(3 * 5, 5 * 7)

In [ ]:
# test
for n in range(5):
    assert fibonacci(n) == fibonacci_iteration(n)

**What are the benefits of recursion?**

- Recursion is often shorter and easier to understand.
- It can be easier to write code by *wishful thinking* or *[declarative programming](https://en.wikipedia.org/wiki/Declarative_programming)* as supposed to [imperative programming](https://en.wikipedia.org/wiki/Imperative_programming).

**Is recusion more efficient than iteration?**

::::{exercise}
:label: ex:recur-vs-iter 

Find the smallest values of `n` for `fibonacci(n)` and `fibonacci_iteration(n)` respectively to run for more than a second.

::::

In [ ]:
%%timeit -n 1
# Assign n the appropriate value
# YOUR CODE HERE
raise NotImplementedError()
fibonacci(n)

In [ ]:
%%timeit -n 1
# Assign n
# YOUR CODE HERE
raise NotImplementedError()
fibonacci_iteration(n)

To understand the difference in performance, modify `fibonacci` to print each function call as follows.

In [ ]:
def fibonacci_verbose(n):
    """Returns the Fibonacci number of order n."""
    print(f"fibonacci({n})")
    return fibonacci_verbose(n - 1) + fibonacci_verbose(n - 2) if n > 1 else 1 if n == 1 else 0


fibonacci_verbose(5)

There are many redundant computations. E.g., `fibonacci(3)` is called twice because

- `fibonacci(5)` calls `fibonacci(4)` and `fibonacci(3)`.
- `fibonacci(4)` then calls `fibonacci(3)` and `fibonacci(2)`.

## Global Variables and Closures

Consider generating a sequence of Fibonacci numbers:

In [ ]:
for n in range(5):
    print(fibonacci_iteration(n))

::::{exercise}
:label: ex:efficient 

Is the above loop efficient?

::::

YOUR ANSWER HERE

**How to avoid redundant computations?**

One way is to store the last two computed Fibonacci numbers.

In [ ]:
%%optlite -r -h 650
Fn, Fnn, n = 0, 1, 0  # global variables


def print_fibonacci_state():
    print(
        f"""Global states:
    Fn  : Next Fibonacci number      = {Fn}
    Fnn : Next next Fibonacci number = {Fnn}
    n   : Next order                 = {n}"""
    )


def next_fibonacci():
    """Returns the next Fibonacci number."""
    global Fn, Fnn, n  # global declaration
    value, Fn, Fnn, n = Fn, Fnn, Fn + Fnn, n + 1
    return value


for i in range(5):
    print(next_fibonacci())
print_fibonacci_state()

::::{important}

Rules for [*global/local variables*](https://docs.python.org/3/faq/programming.html#what-are-the-rules-for-local-and-global-variables-in-python):
1. A local variable must be defined within a function.
1. An assignment defines a local variable except after a [`global` statement](https://docs.python.org/3/reference/simple_stmts.html#the-global-statement).
::::

**Why `global` is NOT needed in `print_fibonacci_state`?**

Without ambiguity, `Fn, Fnn, n` in `print_fibonacci_state` are not local variables by Rule 1 because they are not defined within the function.

**Why `global` is needed in `next_fibonacci`?**

What would happen if the `global` statement is removed:

In [ ]:
%%optlite -h 400
def next_fibonacci():
    """Returns the next Fibonacci number."""
    # global Fn, Fnn, n
    value = n
    n, Fnn, n = Fnn, n + Fnn, n + 1
    return value


next_fibonacci()

`UnboundLocalError` is raised (as supposed to `NameError`) because

- the assignment in Line 5 defines `n` as a local variable by Rule 2, but
- the assignment in Line 4 references `n`, which is not yet defined at that point.

**Are global variables preferred over local ones?**

Consider rewriting the for loop as a while loop:

In [ ]:
%%optlite -h 650
Fn, Fnn, n = 0, 1, 0  # global variables


def print_fibonacci_state():
    print(
        f"""Global states:
    Fn  : Next Fibonacci number      = {Fn}
    Fnn : Next next Fibonacci number = {Fnn}
    n   : Next order                 = {n}"""
    )


def next_fibonacci():
    """Returns the next Fibonacci number."""
    global Fn, Fnn, n  # global declaration
    value, Fn, Fnn, n = Fn, Fnn, Fn + Fnn, n + 1
    return value


n = 0
while n < 5:
    print(next_fibonacci())
    n += 1
print_fibonacci_state()

::::{exercise}
:label: ex:bug 

Why does the while loop print only 3 numbers instead of 5 Fibonacci numbers?

::::

YOUR ANSWER HERE

To avoid such error, a convention in python is to use a leading underscore for variable names that are [*private*](https://www.python.org/dev/peps/pep-0008) (for internal use):  
> _single_leading_underscore: weak "internal use" indicator. E.g., from M import * does not import objects whose names start with an underscore.

In [ ]:
%%optlite -h 600
_Fn, _Fnn, _n = 0, 1, 0  # global variables


def print_fibonacci_state():
    print(
        f"""Global states:
    _Fn  : Next Fibonacci number      = {_Fn}
    _Fnn : Next next Fibonacci number = {_Fnn}
    _n   : Next order                 = {_n}"""
    )


def next_fibonacci():
    """Returns the next Fibonacci number."""
    global _Fn, _Fnn, _n  # global declaration
    value, _Fn, _Fnn, _n = _Fn, _Fnn, _Fn + _Fnn, _n + 1
    return value


n = 0
while n < 5:
    print(next_fibonacci())
    n += 1
print_fibonacci_state()

::::{important}

Using global variables,
- codes are less predictable, more difficult to reuse/extend, and
- tests cannot be isolated, making debugging difficult.
::::

**Is it possible to store the function states without using global variables?**

We can use nested functions and [`nonlocal` variables](https://docs.python.org/3/reference/simple_stmts.html#grammar-token-nonlocal-stmt).

In [ ]:
def fibonacci_sequence(Fn, Fnn):
    def next_fibonacci():
        """Returns the next (generalized) Fibonacci number starting with
        Fn and Fnn as the first two numbers."""
        nonlocal Fn, Fnn, n  # declare nonlocal variables
        value = Fn
        Fn, Fnn, n = Fnn, Fn + Fnn, n + 1
        return value

    def print_fibonacci_state():
        print(
            """States:
        Next Fibonacci number      = {}
        Next next Fibonacci number = {}
        Next order                 = {}""".format(
                Fn, Fnn, n
            )
        )

    n = 0  # Fn and Fnn specified in the function arguments
    return next_fibonacci, print_fibonacci_state


next_fibonacci, print_fibonacci_state = fibonacci_sequence(0, 1)
n = 0
while n < 5:
    print(next_fibonacci())
    n += 1
print_fibonacci_state()

The state variables `Fn, Fnn, n` are now [*encapsulated*](https://en.wikipedia.org/wiki/Encapsulation_(computer_programming)), and the functions returned by `fibonacci_sequence` no longer depend on any global variables.

Another benefit of using nested functions is creating different Fibonacci sequences with different base cases.

In [ ]:
my_next_fibonacci, my_print_fibonacci_state = fibonacci_sequence("cs", "1302")
for n in range(5):
    print(my_next_fibonacci())
my_print_fibonacci_state()

`next_fibonacci` and `print_fibonacci_state` are *local functions* of `fibonacci_sequence`.  

::::{important}
- Local functions can access (*capture*) the other local variables of `fibonacci_sequence` by forming the so-called *closures*.
- Similar to the `global` statement, a [`nonlocal` statement](https://docs.python.org/3/reference/simple_stmts.html#the-nonlocal-statement) is needed for assigning non-local variables.
::::

Each local function has an attribute named `__closure__` that stores the captured local variables.

In [ ]:
def print_closure(f):
    """Print the closure of a function."""
    print("closure of ", f.__name__)
    for cell in f.__closure__:
        print("    {} content: {!r}".format(cell, cell.cell_contents))


print_closure(next_fibonacci)
print_closure(print_fibonacci_state)

## Generator

An easier way to generate a sequence of objects one by one is to write a *generator*.

In [ ]:
fibonacci_generator = (fibonacci_iteration(n) for n in range(3))
fibonacci_generator

The above uses a [*generator expression*](https://docs.python.org/3/reference/expressions.html#grammar-token-generator-expression) to define `fibonacci_generator`.

**How to obtain items from a generator?**

We can use the [`next` function](https://docs.python.org/3/library/functions.html#next).

In [ ]:
fibonacci_generator = (fibonacci_iteration(n) for n in range(3))

while True:
    print(next(fibonacci_generator))  # raises StopIterationException eventually

A generator object is [*iterable*](https://www.programiz.com/python-programming/iterator), i.e., it implements both `__iter__` and `__next__` methods that are automatically called in a `for` loop as well as the `next` function.

In [ ]:
fibonacci_generator = (fibonacci_iteration(n) for n in range(5))

for fib in fibonacci_generator:  # StopIterationException handled by for loop
    print(fib)

**Is `fibonacci_generator` efficient?**

No, again, due to redundant computations. A better way to define the generator is to use the keyword [`yield`](https://docs.python.org/3/reference/expressions.html?highlight=yield#yield-expressions):

In [ ]:
%%optlite -h 450
def fibonacci_sequence(Fn, Fnn, stop):
    """Return a generator that generates Fibonacci numbers
    starting from Fn and Fnn until stop (exclusive)."""
    while Fn < stop:
        yield Fn  # return Fn and pause execution
        Fn, Fnn = Fnn, Fnn + Fn


for fib in fibonacci_sequence(0, 1, 5):
    print(fib)

::::{important}

1. `yield` causes the function to return a *generator* without executing the function body.
1. Calling `__next__` resumes the execution, which 
    - pauses at the subsequent `yield` expression, if any, or
    - raises the `StopIterationException` at the end otherwise.
::::

::::{exercise}
:label: ex:doc 

`yield` can be both a statement and an expression. As an expression: 
- The value of a `yield` expression is `None` by default, but 
- it can be set by the `generator.send` method.

Add the document string to the following function. In particular, explain the effect of calling the method `send` on the returned generator.

::::

In [ ]:
def fibonacci_sequence(Fn, Fnn, stop):
    # YOUR CODE HERE
    raise NotImplementedError()
    while Fn < stop:
        value = yield Fn
        if value is not None:
            Fnn = value  # set next number to the value of yield expression
        Fn, Fnn = Fnn, Fnn + Fn


fibonacci_generator = fibonacci_sequence(0, 1, 5)
print(next(fibonacci_generator))
print(fibonacci_generator.send(2))
for fib in fibonacci_generator:
    print(fib)